In [ ]:
# DIT SCRIPT IS ECHT BIZAR KAKA 
# NIET GEBRUIKEN



import pandas as pd
from datetime import timedelta, datetime
from collections import defaultdict
import random
import math

def generate_initial_solution(dienstregeling, afstandsmatrix):
    """
    Generate an initial feasible solution for the bus planning problem.
    """
    # Standardize location names to lowercase for consistency
    dienstregeling['startlocatie'] = dienstregeling['startlocatie'].str.lower()
    dienstregeling['eindlocatie'] = dienstregeling['eindlocatie'].str.lower()
    afstandsmatrix['startlocatie'] = afstandsmatrix['startlocatie'].str.lower()
    afstandsmatrix['eindlocatie'] = afstandsmatrix['eindlocatie'].str.lower()

    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Add a new 'omloop nummer' column and initialize it
    planning_data['omloop nummer'] = range(1, len(planning_data) + 1)

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'ehvgar')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    # Define the date for all trips in the schedule
    schedule_date = "2024-08-29"

    for _, row in planning_data.iterrows():
        # Implement the original logic from generate_feasible_plan here
        # ...
        pass

    # Combine the bus schedules and charging schedules
    for bus, charging_trips in bus_charging.items():
        bus_schedules.extend(charging_trips)

    # Convert the bus_schedules list to a DataFrame for output
    initial_solution = pd.DataFrame(bus_schedules)
    return initial_solution
def generate_initial_solution(dienstregeling, afstandsmatrix):
    """
    Generate an initial feasible solution for the bus planning problem.
    """
    # Standardize location names to lowercase for consistency
    dienstregeling['startlocatie'] = dienstregeling['startlocatie'].str.lower()
    dienstregeling['eindlocatie'] = dienstregeling['eindlocatie'].str.lower()
    afstandsmatrix['startlocatie'] = afstandsmatrix['startlocatie'].str.lower()
    afstandsmatrix['eindlocatie'] = afstandsmatrix['eindlocatie'].str.lower()

    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Debugging: Check the merged planning data
    print("Merged planning data:")
    print(planning_data.head())  # Print the first few rows to verify the merge

    # Add a new 'omloop nummer' column and initialize it
    planning_data['omloop nummer'] = range(1, len(planning_data) + 1)

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Debugging: Check if the sorting and omloop nummer assignment worked
    print("Sorted and processed planning data:")
    print(planning_data.head())  # Print the first few rows to verify sorting

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'ehvgar')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    # Define the date for all trips in the schedule
    schedule_date = "2024-08-29"

    # Check if the planning_data is empty or has no trips
    if planning_data.empty:
        print("Warning: No trips found in the planning data!")
        return pd.DataFrame()  # Return an empty DataFrame if no data is found

    for _, row in planning_data.iterrows():
        # Implement the original logic from generate_feasible_plan here
        # For now, just add a placeholder trip
        bus_schedules.append({
            'buslijn': row['buslijn'],
            'startlocatie': row['startlocatie'],
            'eindlocatie': row['eindlocatie'],
            'vertrektijd': row['vertrektijd'],
            'omloop nummer': row['omloop nummer']
        })

    # Convert the bus_schedules list to a DataFrame for output
    initial_solution = pd.DataFrame(bus_schedules)

    # Debugging: Check the generated initial solution
    print("Generated initial solution:")
    print(initial_solution.head())  # Print the first few rows to verify

    return initial_solution

def calculate_energy_consumption(distance):
    """
    Calculate the energy consumption for a bus trip based on the distance.
    """
    return distance * 2.5 / 1000  # kWh per meter

def calculate_solution_cost(solution):
    """
    Calculate the cost of a given solution, which is the number of unique omloop nummers.
    """
    if isinstance(solution, pd.DataFrame):
        if 'omloop nummer' in solution.columns:
            return len(set(solution['omloop nummer']))
        else:
            return 0
    else:
        return len(set([trip['omloop nummer'] for trip in solution]))

def simulate_annealing(dienstregeling, afstandsmatrix, initial_temperature, cooling_rate, max_iterations):
    """
    Perform simulated annealing optimization to find a near-optimal solution for the bus planning problem.
    """
    # Generate an initial feasible solution
    current_solution = generate_initial_solution(dienstregeling, afstandsmatrix)
    try:
        current_cost = calculate_solution_cost(current_solution)
    except KeyError as e:
        print(f"KeyError occurred: {e}")
        print("Current solution DataFrame:")
        print(current_solution)
        raise

    best_solution = current_solution
    best_cost = current_cost

    temperature = initial_temperature

    for _ in range(max_iterations):
        # Generate a neighbor solution
        neighbor_solution = generate_neighbor_solution(current_solution)
        neighbor_cost = calculate_solution_cost(neighbor_solution)

        # Calculate the cost difference
        cost_difference = neighbor_cost - current_cost

        # Accept the neighbor solution with a certain probability
        if cost_difference < 0 or random.uniform(0, 1) < math.exp(-cost_difference / temperature):
            current_solution = neighbor_solution
            current_cost = neighbor_cost

            # Update the best solution if necessary
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        # Decrease the temperature
        temperature *= cooling_rate

    return best_solution

def generate_neighbor_solution(current_solution):
    """
    Generate a neighbor solution by swapping the assignment of two trips to different buses.
    """
    # Create a copy of the current solution to modify
    neighbor_solution = current_solution.copy()

    # Check if there are enough trips to sample
    if len(neighbor_solution) < 2:
        # If there are fewer than 2 trips, just return the current solution (no changes)
        return neighbor_solution

    # Randomly select two trips to swap
    trips = neighbor_solution.sample(2)

    # Swap the 'omloop nummer' values for the two trips
    trips['omloop nummer'] = trips['omloop nummer'].iloc[::-1]

    # Replace the original trips with the swapped trips
    for index, row in trips.iterrows():
        neighbor_solution.loc[index, 'omloop nummer'] = row['omloop nummer']

    return neighbor_solution

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.

    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

initial_temperature = 1000
cooling_rate = 0.95
max_iterations = 10000

optimized_solution = simulate_annealing(dienstregeling, afstandsmatrix, initial_temperature, cooling_rate, max_iterations)

# Save the optimized solution to an Excel file
save_plan_to_excel(optimized_solution, 'optimized_plan_consolidated.xlsx')


Merged planning data:
  startlocatie vertrektijd eindlocatie  buslijn  min reistijd in min  \
0       ehvapt       06:04      ehvbst      401                   22   
1       ehvapt       06:34      ehvbst      401                   22   
2       ehvapt       06:56      ehvbst      401                   22   
3       ehvapt       07:11      ehvbst      401                   22   
4       ehvapt       07:26      ehvbst      401                   22   

   max reistijd in min  afstand in meters  
0                   25               9050  
1                   25               9050  
2                   25               9050  
3                   25               9050  
4                   25               9050  
Sorted and processed planning data:
    startlocatie vertrektijd eindlocatie  buslijn  min reistijd in min  \
85        ehvapt       00:01      ehvbst      401                   22   
251       ehvbst       00:09      ehvapt      401                   22   
86        ehvapt       